In [1]:
import pymysql
import pandas as pd
import numpy as np
import random

In [44]:
try:
    # MySQL 데이터베이스에 연결
    conn = pymysql.connect(
        host='capstone-db.cfgmik22w69x.ap-northeast-2.rds.amazonaws.com',
        user='root',
        password='daejin2019',
        db='capstone',
        charset='utf8',
        port=3306
    )
    
    if conn.open:  # is_connected() 대신 open 속성을 사용
        print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")

except pymysql.MySQLError as e:
    print(f"연결 중 오류 발생: {e}")

# finally:
#     if 'connection' in locals() and connection.open:
#         connection.close()
#         print("MySQL 데이터베이스와의 연결이 종료되었습니다.")


MySQL 데이터베이스에 성공적으로 연결되었습니다.


In [3]:
cur = conn.cursor()

cur.execute("CREATE TABLE Accommodation (contentid INT PRIMARY KEY,title VARCHAR(255) NOT NULL, part VARCHAR(25) NOT NULL,area VARCHAR(255) NOT NULL,sigungu VARCHAR(255) NOT NULL,addr VARCHAR(255) NOT NULL,tel VARCHAR(20),firstimage VARCHAR(255),firstimage2 VARCHAR(255))CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")

0

In [4]:
# CSV 파일 경로
csv_file_path = '../data/hotel_after.csv'

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

In [5]:
for index, row in data.iterrows():
    sql = """
        INSERT INTO Accommodation (contentid, title, part, area, sigungu, addr, tel, firstimage, firstimage2)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # NaN 값을 빈 문자열로 대체
    values = [value if pd.notna(value) else '' for value in row.values]
    print(values)  # 디버깅용 출력
    cur.execute(sql, values)

[2465071, '가경재 ', '한옥', '경상북도', '안동시', '경상북도 안동시 하회남촌길 69-5', '054-855-8552', 'http://tong.visitkorea.or.kr/cms/resource/00/2626200_image2_1.jpg', 'http://tong.visitkorea.or.kr/cms/resource/00/2626200_image3_1.jpg']
[142785, '가락관광호텔', '관광호텔', '서울특별시', '송파구', '서울특별시 송파구 송파대로28길 5', '02-400-6641~3', '', '']
[2671267, '가락청', '한옥', '전북특별자치도', '전주시', '전북특별자치도 전주시 완산구 한지길 68', '', '', '']
[2627867, '가람나무', '한옥', '경기도', '파주시', '경기도 파주시 소라지로327번길 126-21', '', '', '']
[1865597, '가람초연재', '한옥', '경상북도', '안동시', '경상북도 안동시 풍천면 하회종가길 76-6', '010-3849-7542', 'http://tong.visitkorea.or.kr/cms/resource/48/2993048_image2_1.jpg', 'http://tong.visitkorea.or.kr/cms/resource/48/2993048_image3_1.jpg']
[1896032, '가름게스트하우스', '펜션', '제주특별자치도', '서귀포시', '제주특별자치도 서귀포시 법환하로9번길 10', '064-739-4499', '', '']
[2571998, '가리벨리 관광펜션', '펜션', '강원특별자치도', '인제군', '강원특별자치도 인제군 인제읍 대목리길 137', '033-463-1212', 'http://tong.visitkorea.or.kr/cms/resource/37/2571137_image2_1.jpeg', 'http://tong.visitkorea.or.kr/cms/resource/37/2571137_i

In [50]:
cur.execute("SELECT COUNT(*) FROM Accommodation;")
a = cur.fetchall()
a

((3829,),)

In [9]:
conn.commit()

### room 전처리 코드

In [48]:
df = pd.read_csv('../data/room_detail_raw.csv')

df.rename(columns={
    'availablecount': 'available_count',
    'roombasecount': 'base_person',
    'roommaxcount': 'max_person',
    'roomoffseasonminfee1': 'offseasonminnfee',
    'roomoffseasonminfee2': 'offseasonmaxnfee',
    'roompeakseasonminfee1': 'peakseasonminfee',
    'roompeakseasonminfee2': 'peakseasonmaxfee',
    'roomimg1alt': 'roomimg1_alt',
    'roomimg2alt': 'roomimg2_alt',
    'roomimg3alt': 'roomimg3_alt'
}, inplace=True)

df = df.drop_duplicates(subset=['contentid', 'roomcode'])
df['roomcount'] = np.where(df['roomcount'] == 0, np.random.randint(0, 11, size=len(df)), df['roomcount'])

# 이미 존재하는 roomcode와 contentid 목록 생성
existing_roomcodes = set(df['roomcode'].dropna().unique())
existing_contentids = set(df['contentid'].dropna().unique())

# roomcode가 비어있는 경우 처리
for index, row in df.iterrows():
    if pd.isnull(row['roomcode']) or row['roomcode'] == '':
        while True:
            # 5~6자리 숫자 랜덤 roomcode 생성
            length = random.randint(5, 6)
            new_roomcode = ''.join(random.choices('0123456789', k=length))
            
            # 새 roomcode가 기존 roomcode와 contentid에 없는지 확인
            if new_roomcode not in existing_roomcodes and new_roomcode not in existing_contentids:
                df.at[index, 'roomcode'] = new_roomcode  # 새로운 roomcode를 데이터프레임에 추가
                existing_roomcodes.add(new_roomcode)  # 새 roomcode를 기존 목록에 추가
                break  # 중복되지 않으면 루프 종료

df['roomcount'] = np.where(df['roomcount'] == 0, np.random.randint(0, 11, size=len(df)), df['roomcount'])

# available_count 계산: room_count에서 0부터 5까지 무작위로 뺀 값을 넣습니다.
df['available_count'] = np.where(df['roomcount'] > 0,df['roomcount'] - np.random.randint(0, df['roomcount'] + 1, size=len(df)), 0)
df['base_person'] = np.where(df['base_person'] == 0, np.random.randint(0, 25, size=len(df)), df['base_person'])
df['max_person'] = df['base_person'] + np.random.randint(0, 6, size=len(df))
df['offseasonminnfee'] = np.where(df['offseasonminnfee'] == 0,np.random.randint(200, 1000, size=len(df)) * 100,  df['offseasonminnfee'])
df['offseasonmaxnfee'] = np.where((df['offseasonmaxnfee'].isnull()) | (df['offseasonmaxnfee'] == 0) | (df['offseasonmaxnfee'] == df['offseasonminnfee']),df['offseasonmaxnfee'] + np.random.randint(5, 50, size=len(df)) * 1000,df['offseasonmaxnfee'])
df['peakseasonminfee'] = np.where(df['peakseasonminfee'] == 0,np.random.randint(200, 1000, size=len(df)) * 100,  df['peakseasonminfee'])
df['peakseasonmaxfee'] = np.where((df['peakseasonmaxfee'].isnull()) | (df['peakseasonmaxfee'] == 0) | (df['peakseasonmaxfee'] == df['peakseasonminfee']),df['peakseasonmaxfee'] + np.random.randint(5, 50, size=len(df)) * 1000,df['offseasonmaxnfee'])
drop_columns = ['contenttypeid', 'roomsize1', 'roomintro', 'cpyrhtDivCd1','cpyrhtDivCd2','cpyrhtDivCd3','roomimg4','roomimg4alt','cpyrhtDivCd4','roomimg5','roomimg5alt','cpyrhtDivCd5','roomsize2']
df = df.drop(columns=drop_columns)
df.fillna('', inplace=True)
new_order = [
    'contentid', 'roomcode', 'roomtitle', 'roomcount', 'available_count',
    'base_person', 'max_person', 'offseasonminnfee', 'offseasonmaxnfee',
    'peakseasonminfee', 'peakseasonmaxfee', 'roombathfacility', 'roombath',
    'roomhometheater', 'roomaircondition', 'roomtv', 'roompc', 'roomcable',
    'roominternet', 'roomrefrigerator', 'roomtoiletries', 'roomsofa',
    'roomcook', 'roomtable', 'roomhairdryer', 'roomimg1', 'roomimg1_alt',
    'roomimg2', 'roomimg2_alt', 'roomimg3', 'roomimg3_alt'
]

# 데이터프레임의 열 순서 변경
df = df[new_order]

df.to_csv('../data/room.csv', index=False, encoding='utf-8-sig')

C:\Users\cho03\AppData\Local\Temp\ipykernel_23232\841627387.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '622377' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'roomcode'] = new_roomcode  # 새로운 roomcode를 데이터프레임에 추가


In [49]:
cur = conn.cursor()

cur.execute("CREATE TABLE Room (contentid INT NOT NULL,roomcode INT NOT NULL,roomtitle VARCHAR(255) NOT NULL,roomcount INT,available_count INT NOT NULL,base_person INT NOT NULL,max_person INT NOT NULL,offseasonminnfee INT,offseasonmaxnfee INT,peakseasonminfee INT,peakseasonmaxfee INT,roombathfacility VARCHAR(1),roombath VARCHAR(1),roomhometheater VARCHAR(1),roomaircondition VARCHAR(1),roomtv VARCHAR(1),roompc VARCHAR(1),roomcable VARCHAR(1),roominternet VARCHAR(1),roomrefrigerator VARCHAR(1),roomtoiletries VARCHAR(1),roomsofa VARCHAR(1),roomcook VARCHAR(1),roomtable VARCHAR(1),roomhairdryer VARCHAR(1),rooming1 VARCHAR(1),rooming1_alt VARCHAR(1),rooming2 VARCHAR(1),rooming2_alt VARCHAR(1),rooming3 VARCHAR(1),rooming3_alt VARCHAR(1),PRIMARY KEY (contentid, roomcode));")
conn.commit()

In [51]:
# CSV 파일 경로
csv_file_path = '../data/room.csv'

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

In [52]:
for index, row in data.iterrows():
    sql = """
        INSERT INTO Room (contentid, roomcode, roomtitle, roomcount, available_count, base_person, max_person, offseasonminnfee, offseasonmaxnfee, peakseasonminfee, peakseasonmaxfee, roombathfacility,roombath, roomhometheater, roomaircondition, roomtv, roompc, roomcable,roominternet,roomrefrigerator,roomtoiletries,roomsofa, roomcook, roomtable,roomhairdryer, rooming1,rooming1_alt, rooming2, rooming2_alt, rooming3, rooming3_alt)        
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # NaN 값을 빈 문자열로 대체
    values = [value if pd.notna(value) else '' for value in row.values]
    print(values)  # 디버깅용 출력
    cur.execute(sql, values)
conn.commit()

[2465071, 46591.0, '사랑채 아랫방', 2, 0, 2, 2, 80000, 117000, 80000, 113000, '', '', '', 'Y', 'Y', '', 'Y', 'Y', 'Y', 'Y', '', '', 'Y', 'Y', 'http://tong.visitkorea.or.kr/cms/resource/16/2626216_image2_1.jpg', '경북_안동_가경재_사랑채(아랫방)03', 'http://tong.visitkorea.or.kr/cms/resource/15/2626215_image2_1.jpg', '경북_안동_가경재_사랑채(아랫방)02', 'http://tong.visitkorea.or.kr/cms/resource/14/2626214_image2_1.jpg', '경북_안동_가경재_사랑채(아랫방)01']
[2465071, 46593.0, '군불 황토 아랫방', 7, 2, 3, 6, 100000, 115000, 100000, 115000, '', '', '', 'Y', 'Y', '', 'Y', 'Y', 'Y', 'Y', '', '', 'Y', 'Y', 'http://tong.visitkorea.or.kr/cms/resource/23/2626223_image2_1.jpg', '경북_안동_가경재_군불황토(아랫방)05', 'http://tong.visitkorea.or.kr/cms/resource/22/2626222_image2_1.jpg', '경북_안동_가경재_군불황토(아랫방)04', 'http://tong.visitkorea.or.kr/cms/resource/19/2626219_image2_1.jpg', '경북_안동_가경재_군불황토(아랫방)01']
[142785, 11432.0, '스탠다드 더블', 2, 2, 2, 5, 98000, 130000, 98000, 107000, 'Y', 'Y', '', 'Y', 'Y', '', 'Y', 'Y', 'Y', 'Y', '', '', '', 'Y', '', '', '', '', '', '']
[14

In [53]:
cur.execute("SELECT COUNT(*) FROM Room;")
a = cur.fetchall()
a

((11752,),)

In [54]:
conn.commit()
conn.close()
print("MySQL 데이터베이스와의 연결이 종료되었습니다.")

MySQL 데이터베이스와의 연결이 종료되었습니다.
